In [4]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import random
import time
import itertools
import math
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [5]:
LENGTH=100
WIDE=50

time_layout1 = pd.DataFrame({
    'L/U':[0,12,10,8,6],
    'M1':[6,0,6,8,10],
    'M2':[8,6,0,6,8],
    'M3':[10,8,6,0,6],
    'M4':[12,10,8,6,0]},
    index = ['L/U','M1','M2','M3','M4'])

time_layout2 = pd.DataFrame({
    'L/U':[0,6,8,6,4],
    'M1':[4,0,12,10,8],
    'M2':[6,2,0,12,10],
    'M3':[8,4,2,0,12],
    'M4':[6,2,4,2,0]},
    index = ['L/U','M1','M2','M3','M4'])

time_layout3 = pd.DataFrame({
    'L/U':[0,12,10,4,2],
    'M1':[2,0,12,6,4],
    'M2':[4,2,0,8,6],
    'M3':[10,8,6,0,12],
    'M4':[12,10,8,2,0]},
    index = ['L/U','M1','M2','M3','M4'])

time_layout4 = pd.DataFrame({
    'L/U':[0,18,20,12,14],
    'M1':[4,0,14,8,14],
    'M2':[8,4,0,6,12],
    'M3':[10,6,8,0,6],
    'M4':[14,10,6,6,0]},
                            
    index = ['L/U','M1','M2','M3','M4'])

gantt_chart = pd.DataFrame({
    'M1':np.zeros(LENGTH),
    'M2':np.zeros(LENGTH),
    'M3':np.zeros(LENGTH),
    'M4':np.zeros(LENGTH)},
    index = [np.arange(1,LENGTH+1)],dtype=int)
#print(gantt_chart)

operation = pd.DataFrame({
    'job':np.zeros(WIDE),
    'process':np.zeros(WIDE)},
    index = [np.arange(1,WIDE+1)],dtype=int)

state_M=np.zeros(4,dtype=int)
state_J1=np.zeros(5,dtype=int)
state_J2=np.zeros(6,dtype=int)
state_J3=np.zeros(6,dtype=int)
state_J4=np.zeros(5,dtype=int)
state_J5=np.zeros(5,dtype=int)
state_J6=np.zeros(6,dtype=int)
state_J7=np.zeros(8,dtype=int)
state_J8=np.zeros(6,dtype=int)
state_J9=np.zeros(5,dtype=int)
state_J10=np.zeros(6,dtype=int)


In [6]:
#JOB1

o=1#operation迭代
best_t=9999

state_M=np.zeros(4,dtype=int)
state_J1=np.zeros(5,dtype=int)

gantt_chart = pd.DataFrame({
    'M1':np.zeros(LENGTH),
    'M2':np.zeros(LENGTH),
    'M3':np.zeros(LENGTH),
    'M4':np.zeros(LENGTH)},
    index = [np.arange(1,LENGTH+1)],dtype=int)
#print(gantt_chart)

operation = pd.DataFrame({
    'job':np.zeros(WIDE),
    'process':np.zeros(WIDE)},
    index = [np.arange(1,WIDE+1)],dtype=int)

MS_J1=np.array([[1,2,4],
               [1,3,2],
               [3,4,1],
               [4,2,-1],
               [3,1,-1]])#-1は作業なしと表示される

T_J1=np.array([[8,16,12],
               [20,10,18],
               [12,8,15],
               [14,18,-1],
               [10,15,-1]])#-1は作業なしと表示される

check_J1=np.array([[0,0,0],
                   [0,0,0],
                   [0,0,0],
                   [0,0,-1],
                   [0,0,-1]]) #ジョブの確認表（0:まだしていない、1：もうした、-1：作業なし）

gantt_chart_J1=gantt_chart
operation_J1=operation

list_arrange_J1 = list(itertools.permutations(np.arange(0,5)))
#print(list_arrange_J1)
list_time_J1=np.zeros(math.factorial(5))
gene=0

for a in list_arrange_J1:
    #print('现在的顺序为：',a)
    gantt_chart_J1=gantt_chart
    operation_J1=operation
    for t in np.arange(1,LENGTH+1):#時系列
        if np.any(state_M==0) and np.any(state_J1==0):#空いている機械と空いているジョブはあるかどうかを検査
            for j in a:#ジョブ1からジョブ５までループ
                if state_J1[j]==0:#選んだジョブｊは空いているかどうかを検査
                    for i in np.arange(0,3):#選んだジョブｊの中のプロセスｊをループ
                        if check_J1[j,i]==0 and state_J1[j]==0:#選んだプロセス(i,j)はしたかどうかを検査、また循環の中でジョブｊは空いているかどうかを確認
                            if i==0 or (i>0 and check_J1[j,i-1]==1):#選んだ空いているプロセス(i,j)は手前のプロセスや前のプロセスはもうしたの場合
                                m=MS_J1[j,i]-1#プロセス(i,j)の作業機械番号を代入
                                if state_M[m]==0:#プロセス(i,j)の作業機械は空いているかどうかを確認
                                    state_M[m]=T_J1[j,i]#作業時間を機械状態に代入
                                    state_J1[j]=T_J1[j,i]#作業時間をジョブ状態に代入
                                    check_J1[j,i]=1#プロセス(i,j)にチャックを付く
                                    gantt_chart_J1.iloc[t-1:t-1+T_J1[j,i],m]=j+1#作業時間とジョブ番号を代入
                                    operation_J1.loc[o,'job']=j+1
                                    operation_J1.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1#カウントダウン
        state_J1[state_J1 !=0] -= 1
        if np.all(check_J1!=0):#チェック表をチェック、作業全部済んだら、プールから出る
            break
   
    for t in np.arange(1,LENGTH+1):
        if np.all(gantt_chart_J1.iloc[t-1,:]==0):    
               #print('ジョブ完成時刻:第',t,'時刻')
               break
    
    list_time_J1[gene]=t
    gene += 1
    
    if t < best_t:
        best_t_J1=t
        best_gantt_chart_J1=gantt_chart_J1
        best_operation_J1=operation_J1

print(list_time_J1)
print('ジョブ1の最適解の作業完成時刻:第',best_t_J1,'時刻')
print(best_gantt_chart_J1)
print(best_operation_J1)        
        
raw_data1=best_gantt_chart_J1
np.savetxt(fname='d:/pysrc/data/gantt_chart_J1.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J1
np.savetxt(fname='d:/pysrc/data/operation_J1.csv',X=raw_data2,fmt='%0d',delimiter=',')

[61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.
 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.
 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.
 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.
 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.
 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.
 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61. 61.]
ジョブ1の最適解の作業完成時刻:第 61 時刻
     M1  M2  M3  M4
1     1   0   3   4
2     1   0   3   4
3     1   0   3   4
4     1   0   3   4
5     1   0   3   4
6     1   0   3   4
7     1   0   3   4
8     1   0   3   4
9     2   1   3   4
10    2   1   3   4
11    2   1   3   4
12    2   1   3   4
13    2   1   5   4
14    2   1   5   4
15    2   1   5   3
16    2   1   5   3
17    2   1   5   3
18    2   1   5   3
19    2   1   5   3
20    2   1   5   3
21    2   1   5   3
22    2   1   5   3
23    2   1   0   0
24    2 

In [7]:
#JOB2

o=1#operation迭代
best_t=9999

state_M=np.zeros(4,dtype=int)
state_J2=np.zeros(6,dtype=int)

gantt_chart = pd.DataFrame({
    'M1':np.zeros(LENGTH),
    'M2':np.zeros(LENGTH),
    'M3':np.zeros(LENGTH),
    'M4':np.zeros(LENGTH)},
    index = [np.arange(1,LENGTH+1)],dtype=int)
#print(gantt_chart)

operation = pd.DataFrame({
    'job':np.zeros(WIDE),
    'process':np.zeros(WIDE)},
    index = [np.arange(1,WIDE+1)],dtype=int)

MS_J2=np.array([[1,4,-1],
               [2,4,-1],
               [1,3,-1],
               [2,3,4],
               [1,2,4],
               [1,2,3]])

T_J2=np.array([[10,18,-1],
               [10,18,-1],
               [10,20,-1],
               [10,15,12],
               [10,15,12],
               [10,15,12]])

check_J2=np.array([[0,0,-1],
                   [0,0,-1],
                   [0,0,-1],
                   [0,0,0],
                   [0,0,0],
                   [0,0,0]]) 

gantt_chart_J2=gantt_chart
operation_J2=operation

list_arrange_J2 = list(itertools.permutations(np.arange(0,6)))
list_time_J2=np.zeros(math.factorial(6))
gene=0

for a in list_arrange_J2:
    #print('现在的顺序为：',a)
    
    gantt_chart_J2=gantt_chart
    operation_J2=operation
    
    for t in np.arange(1,LENGTH+1):
        if np.any(state_M==0) and np.any(state_J2==0):
            for j in a:
                if state_J2[j]==0:
                    for i in np.arange(0,3):
                        if check_J2[j,i]==0 and state_J2[j]==0:
                            if i==0 or (i>0 and check_J2[j,i-1]==1):
                                m=MS_J2[j,i]-1
                                if state_M[m]==0:
                                    state_M[m]=T_J2[j,i]
                                    state_J2[j]=T_J2[j,i]
                                    check_J2[j,i]=1
                                    gantt_chart_J2.iloc[t-1:t-1+T_J2[j,i],m]=j+1#作業時間とジョブ番号を代入
                                    operation_J2.loc[o,'job']=j+1
                                    operation_J2.loc[o,'process']=i+1
                                    o+=1
        state_M[state_M !=0] -= 1#カウントダウン
        state_J2[state_J2 !=0] -= 1
        if np.all(check_J2!=0):#チェック表をチェック、作業全部済んだら、プールから出る
            break
   
    for t in np.arange(1,LENGTH+1):
        if np.all(gantt_chart_J2.iloc[t-1,:]==0):    
               #print('ジョブ完成時刻:第',t,'時刻')
               break
    
    list_time_J2[gene]=t
    gene += 1
    
    if t < best_t:
        best_t_J2=t
        best_gantt_chart_J2=gantt_chart_J2
        best_operation_J2=operation_J2

print(list_time_J2)
print('ジョブ1の最適解の作業完成時刻:第',best_t_J2,'時刻')
print(best_gantt_chart_J2)
print(best_operation_J2)         
        
raw_data1=best_gantt_chart_J2
np.savetxt(fname='d:/pysrc/data/gantt_chart_J2.csv',X=raw_data1,fmt='%0d',delimiter=',')
raw_data2=best_operation_J2
np.savetxt(fname='d:/pysrc/data/operation_J2.csv',X=raw_data2,fmt='%0d',delimiter=',')

[73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73.
 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73. 73